## Install packages

install required python packages

In [19]:
!pip install --upgrade pip
!pip install psycopg2
!pip install pandas
!pip install openpyxl

1229.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/bin/zsh: /home/dhiman/miniconda3/lib/libncursesw.so.6: no version information available (required by /bin/zsh)


1238.67s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/bin/zsh: /home/dhiman/miniconda3/lib/libncursesw.so.6: no version information available (required by /bin/zsh)


1247.55s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/bin/zsh: /home/dhiman/miniconda3/lib/libncursesw.so.6: no version information available (required by /bin/zsh)


1256.29s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/bin/zsh: /home/dhiman/miniconda3/lib/libncursesw.so.6: no version information available (required by /bin/zsh)


## Import modules

import required python modules for data load

In [20]:
import psycopg2
import pandas as pd
import psycopg2.extras

### PostgreSQL Connector


In [21]:
# sudo -u postgres psql
# create database ecomdb;
# create user dhiman with encrypted password 'dhiman1234';
# grant all privileges on database ecomdb to dhiman;
# \c ecomdb postgres;
# GRANT ALL ON SCHEMA public TO dhiman;

class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "dhiman",
                                           password = "dhiman1234",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        return self.connection

### create Star Schema in the PostgreSQL database

In [22]:
def create_tables(connection):
    commands = (
        """
        CREATE TABLE IF NOT EXISTS trans_dim (
            payment_key VARCHAR(10) PRIMARY KEY,
            trans_type VARCHAR(10) NOT NULL,
            bank_name VARCHAR(100)
        )
        """,
        """ CREATE TABLE IF NOT EXISTS item_dim (
            item_key VARCHAR(10) PRIMARY KEY,
            item_name VARCHAR(255) NOT NULL,
            item_desc VARCHAR(255) NOT NULL,
            unit_price DECIMAL NOT NULL,
            man_country VARCHAR(100) NOT NULL,
            supplier VARCHAR(255) NOT NULL,
            unit VARCHAR(15) NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS customer_dim (
            customer_key VARCHAR(10) PRIMARY KEY,
            customer_name VARCHAR(100) NOT NULL,
            contact_no VARCHAR(20) NOT NULL,
            nid VARCHAR(20) NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS time_dim (
            time_key VARCHAR(10) PRIMARY KEY,
            time_date TIMESTAMP NOT NULL,
            time_hour smallint NOT NULL,
            time_day smallint NOT NULL,
            time_week VARCHAR(15) NOT NULL,
            time_month smallint NOT NULL,
            time_quarter VARCHAR(3) NOT NULL,
            time_year smallint NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS store_dim (
            store_key VARCHAR(10) PRIMARY KEY,
            division VARCHAR(20) NOT NULL,
            district VARCHAR(30) NOT NULL,
            upazila VARCHAR(40) NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS fact_table (
                payment_key VARCHAR(10) NOT NULL,
                customer_key VARCHAR(10) NOT NULL,
                time_key VARCHAR(10) NOT NULL,
                item_key VARCHAR(10) NOT NULL,
                store_key VARCHAR(10) NOT NULL,
                quantity INTEGER NOT NULL,
                unit VARCHAR(15) NOT NULL,
                unit_price DECIMAL NOT NULL,
                total_price DECIMAL NOT NULL,
                PRIMARY KEY (payment_key , customer_key, time_key, item_key, store_key),
                FOREIGN KEY (payment_key)
                    REFERENCES trans_dim (payment_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (customer_key)
                    REFERENCES customer_dim (customer_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (time_key)
                    REFERENCES time_dim (time_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (item_key)
                    REFERENCES item_dim (item_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (store_key)
                    REFERENCES store_dim (store_key)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)

    cur = connection.cursor()
    # create table one by one
    for command in commands:
        cur.execute(command)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    connection.commit()


In [23]:
con = PostgresConnection().getConnection()
create_tables(connection=con)
con.close()

### Read data

read data from xlsx file using panda

In [24]:
trans_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Trans_dim', engine='openpyxl')

trans_dim

,payment_key,trans_type,bank_name
0,P001,cash,NaN
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


In [25]:
item_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Item_dim', engine='openpyxl')

item_dim

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
...,...,...,...,...,...,...,...
259,I00260,Napkins Square Lunch,Kitchen Supplies,15.00,Cambodia,NINGBO SEDUNO IMP & EXP CO.LTD,ct
260,I00261,POM 2 ply paper towels,Kitchen Supplies,33.00,Lithuania,BIGSO AB,ct
261,I00262,Strong Everyday Napkins,Kitchen Supplies,14.00,Netherlands,Bolsius Boxmeer,ct
262,I00263,Advil 2 pill packets,Medicine,14.00,Lithuania,BIGSO AB,Ct


In [26]:
customer_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Coustomer_dim', engine='openpyxl')

customer_dim

In [ ]:
time_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Time_dim', engine='openpyxl')
time_dim['date'] = pd.to_datetime(time_dim['date'], format='%d-%m-%Y %H:%M')
time_dim

In [ ]:
store_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Store_dim', engine='openpyxl')

store_dim

In [ ]:
fact_table = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Fact_table', engine='openpyxl')

fact_table

### Insert data

Insert data into db from pandas

#### Insert into trans_dim table

In [ ]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO trans_dim(payment_key, trans_type, bank_name) VALUES (%s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, trans_dim.values)
con.commit()
cur.close()
con.close()

#### Insert into customer_dim table

In [ ]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO customer_dim(customer_key, customer_name, contact_no, nid) VALUES (%s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, customer_dim.values)
con.commit()
cur.close()
con.close()

#### Insert into item_dim table


In [ ]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO item_dim(item_key, item_name, item_desc, unit_price, man_country, supplier, unit) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, item_dim.values)
con.commit()
cur.close()
con.close()

#### Insert into store_dim table

In [ ]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO store_dim(store_key, division, district, upazila) VALUES (%s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, store_dim.values)
con.commit()
cur.close()
con.close()

#### Insert into time_dim table

In [ ]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO time_dim(time_key, time_date, time_hour, time_day, time_week, time_month, time_quarter, time_year)VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, time_dim.values)
con.commit()
cur.close()
con.close()

#### Insert into fact_table table

In [ ]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO fact_table(payment_key, customer_key, time_key, item_key, store_key, quantity, unit, unit_price, total_price)VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, fact_table.values)
con.commit()
cur.close()
con.close()